In [1]:
# Importing necessary libraries 
import os
import json
import pandas as pd
import numpy as np
import glob
from IPython import get_ipython


# For data cleaning and visualization
import pandas as pd
import numpy as np
from numpy import array
from datetime import date
from datetime import datetime, timedelta
import pandas_datareader.data as web


# For saving the model
import pickle

# For model Evaluation
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
import math

import re
import nltk
from nltk.corpus import stopwords

In [2]:
scores_df = pd.read_csv('FinalData/vadar_scores.csv', index_col=0)
mean_scores = pd.read_csv('FinalData/vadar_mean_compound_scores.csv', index_col=0)

In [3]:
scores_df

,headline,date,neg,neu,pos,compound
0,Nikkei down 1 4 pct on economy fears autos drop,2008-10-02,0.450,0.550,0.000,-0.5994
1,FOREX Dollar poised for biggest weekly gain in...,2008-10-03,0.000,0.597,0.403,0.6597
2,GLOBAL MARKETS Stocks rally on Wells Wachovia ...,2008-10-03,0.000,0.645,0.355,0.5267
3,GLOBAL MARKETS Stocks hold up after Wells Wach...,2008-10-03,0.000,0.800,0.200,0.2500
4,GLOBAL MARKETS U S bailout plan passes inves...,2008-10-03,0.136,0.583,0.282,0.3612
...,...,...,...,...,...,...
221508,Why T Mobile s deal with Sprint could be the w...,2020-02-13,0.000,0.881,0.119,0.2263
221509,6 Low Price to Book Stocks With Great Growth P...,2020-02-13,0.131,0.313,0.556,0.7783
221510,Cisco Systems CSCO Q2 Earnings And Revenues ...,2020-02-13,0.000,1.000,0.000,0.0000
221511,Improve Your Retirement Income With These 3 To...,2020-02-13,0.000,0.701,0.299,0.5719


In [4]:
mean_scores

,mean_compound,date
0,-0.599400,2008-10-02
1,0.449400,2008-10-03
2,-0.655600,2008-10-06
3,-0.460450,2008-10-07
4,0.223500,2008-10-08
...,...,...
3636,0.074910,2020-02-09
3637,0.097086,2020-02-10
3638,0.005268,2020-02-11
3639,0.041805,2020-02-12


In [5]:
# Helper Functions 
def getTickerData(ticker, start, end):
  # exception handling for unexpected errors
  try:
    df = web.DataReader(ticker, 'yahoo', start, end)
    # convert the web data to csv format
    df.to_csv('{}.csv'.format(ticker), index=True)
  except:
    print("Error in getting Data...")
def get_data(start_date, end_date, company, stkMarket=""):
  # Ticker Information  
  # Ticker Examples = ['MMM',  'ABT',  'AAPL',  'AMAT',  'APTV',  'ADM',  'ARNC',  'AMZN', 'GOOG']   
  # Stock data retrieval using yahoo
  ticker = company
  stockMarket = stkMarket
  if stockMarket:
    ticker = ticker + "." + stockMarket
    print("Ticker = ",ticker)
  print(f"Collecting the required data for {ticker} from dates: {start_date} to {end_date}...")
  getTickerData(ticker, start_date, end_date)
  df = web.DataReader(ticker, 'yahoo', start_date, end_date)
  df = pd.read_csv('{}.csv'.format(ticker))
  return df

In [83]:
# setting start and end date for stock data retrieval
company = 'NIO'
start_date = date(2008,1,1)
end_date = date(2021,1,1)

# fetching data using ticker
sdf = get_data(start_date,end_date,company)
sdf

,Date,High,Low,Open,Close,Volume,Adj Close
0,2018-09-12,6.930000,5.350000,6.000000,6.600000,66849000,6.600000
1,2018-09-13,12.690000,6.520000,6.620000,11.600000,158346500,11.600000
2,2018-09-14,13.800000,9.220000,12.660000,9.900000,172473600,9.900000
3,2018-09-17,9.750000,8.500000,9.610000,8.500000,56323900,8.500000
4,2018-09-18,9.100000,7.670000,8.730000,7.680000,41827600,7.680000
...,...,...,...,...,...,...,...
576,2020-12-24,47.130001,45.349998,46.880001,45.770000,60582500,45.770000
577,2020-12-28,46.849998,43.669998,46.669998,44.060001,105078000,44.060001
578,2020-12-29,46.279999,42.230000,43.700001,46.139999,120689900,46.139999
579,2020-12-30,48.380001,45.720001,46.500000,48.380001,116691700,48.380001


In [84]:
sdf = sdf.dropna()
sdf = sdf.reset_index(drop=True)
sdf

,Date,High,Low,Open,Close,Volume,Adj Close
0,2018-09-12,6.930000,5.350000,6.000000,6.600000,66849000,6.600000
1,2018-09-13,12.690000,6.520000,6.620000,11.600000,158346500,11.600000
2,2018-09-14,13.800000,9.220000,12.660000,9.900000,172473600,9.900000
3,2018-09-17,9.750000,8.500000,9.610000,8.500000,56323900,8.500000
4,2018-09-18,9.100000,7.670000,8.730000,7.680000,41827600,7.680000
...,...,...,...,...,...,...,...
576,2020-12-24,47.130001,45.349998,46.880001,45.770000,60582500,45.770000
577,2020-12-28,46.849998,43.669998,46.669998,44.060001,105078000,44.060001
578,2020-12-29,46.279999,42.230000,43.700001,46.139999,120689900,46.139999
579,2020-12-30,48.380001,45.720001,46.500000,48.380001,116691700,48.380001


In [85]:
sdf['Date'] = pd.to_datetime(sdf['Date'])
sdf = sdf.sort_values(by=['Date'], ascending=True)
sdf = sdf.reset_index(drop=True)
sdf

,Date,High,Low,Open,Close,Volume,Adj Close
0,2018-09-12,6.930000,5.350000,6.000000,6.600000,66849000,6.600000
1,2018-09-13,12.690000,6.520000,6.620000,11.600000,158346500,11.600000
2,2018-09-14,13.800000,9.220000,12.660000,9.900000,172473600,9.900000
3,2018-09-17,9.750000,8.500000,9.610000,8.500000,56323900,8.500000
4,2018-09-18,9.100000,7.670000,8.730000,7.680000,41827600,7.680000
...,...,...,...,...,...,...,...
576,2020-12-24,47.130001,45.349998,46.880001,45.770000,60582500,45.770000
577,2020-12-28,46.849998,43.669998,46.669998,44.060001,105078000,44.060001
578,2020-12-29,46.279999,42.230000,43.700001,46.139999,120689900,46.139999
579,2020-12-30,48.380001,45.720001,46.500000,48.380001,116691700,48.380001


In [86]:
mean_scores

,mean_compound,date
0,-0.599400,2008-10-02
1,0.449400,2008-10-03
2,-0.655600,2008-10-06
3,-0.460450,2008-10-07
4,0.223500,2008-10-08
...,...,...
3636,0.074910,2020-02-09
3637,0.097086,2020-02-10
3638,0.005268,2020-02-11
3639,0.041805,2020-02-12


In [87]:
mean_scores['date'] = pd.to_datetime(mean_scores['date'])
mean_scores = mean_scores.sort_values(by=['date'], ascending=True)
mean_scores = mean_scores.reset_index(drop=True)
mean_scores

,mean_compound,date
0,-0.599400,2008-10-02
1,0.449400,2008-10-03
2,-0.655600,2008-10-06
3,-0.460450,2008-10-07
4,0.223500,2008-10-08
...,...,...
3636,0.074910,2020-02-09
3637,0.097086,2020-02-10
3638,0.005268,2020-02-11
3639,0.041805,2020-02-12


In [88]:
sdf.shape, mean_scores.shape

((581, 7), (3641, 2))

In [89]:
sdf = sdf.drop(columns=['High', 'Low','Open','Close','Volume'])
sdf

,Date,Adj Close
0,2018-09-12,6.600000
1,2018-09-13,11.600000
2,2018-09-14,9.900000
3,2018-09-17,8.500000
4,2018-09-18,7.680000
...,...,...
576,2020-12-24,45.770000
577,2020-12-28,44.060001
578,2020-12-29,46.139999
579,2020-12-30,48.380001


In [90]:
sdf['date'] = sdf['Date']
sdf.drop(columns=['Date'],inplace=True)
sdf

,Adj Close,date
0,6.600000,2018-09-12
1,11.600000,2018-09-13
2,9.900000,2018-09-14
3,8.500000,2018-09-17
4,7.680000,2018-09-18
...,...,...
576,45.770000,2020-12-24
577,44.060001,2020-12-28
578,46.139999,2020-12-29
579,48.380001,2020-12-30


In [91]:
merged_inner = pd.merge(left=mean_scores, right=sdf, left_on='date', right_on='date')
merged_inner

,mean_compound,date,Adj Close
0,0.011711,2018-09-12,6.60
1,0.060028,2018-09-13,11.60
2,0.000588,2018-09-14,9.90
3,0.020251,2018-09-17,8.50
4,0.066039,2018-09-18,7.68
...,...,...,...
353,0.027369,2020-02-07,3.81
354,0.097086,2020-02-10,3.87
355,0.005268,2020-02-11,3.96
356,0.041805,2020-02-12,4.07


In [92]:
merged_inner = merged_inner.rename(columns = {'Adj Close': 'adj_close'})
df_final = merged_inner[['date','adj_close','mean_compound']]
df_final

,date,adj_close,mean_compound
0,2018-09-12,6.60,0.011711
1,2018-09-13,11.60,0.060028
2,2018-09-14,9.90,0.000588
3,2018-09-17,8.50,0.020251
4,2018-09-18,7.68,0.066039
...,...,...,...
353,2020-02-07,3.81,0.027369
354,2020-02-10,3.87,0.097086
355,2020-02-11,3.96,0.005268
356,2020-02-12,4.07,0.041805


In [93]:
df_name = "FinalData/stock_and_news_"+company+".csv"
df_final.to_csv(df_name)